# DEL 3.2 - Validering med pydantic

I denne delen skal vi se på biblioteket `pydantic`. Dette er ikke en del av
bibliotekene som leveres sammen med `python`, og må installeres med `pip`.

Kjør cellen under for å importere bibliotekene vi trenger. Dersom du får beskjed
om at `pydantic` ikke ble funnet, følg 
[instruksjonene under](#installere-og-importere-pydantic) 
for å installere biblioteket.

In [ ]:
# Prøver å importere pydantic
try:
    from pydantic import BaseModel
    print("pydantic er importert og klar til bruk!")
except ImportError:
    print("Finner ikke pydantic. " +
          "Følg instruksjonene under for å installere " +
          "pydantic, og kjør denne cellen på nytt.")

## Installere og importere pydantic

> Du kan hoppe over dette om cellen over kjørte og du fikk meldingen
> "pydantic er importert og klar til bruk"

Åpne et terminalvindu, og sør for at du står i mappen til dette prosjektet.
Sjekk videre at du har aktivert `.venv`, som du ser ved at det står (.venv)
foran mappebanen du står i. Hvis ikke du ser (.venv) i terminalen, kan du
aktivere miljøet med kommandoen:

```bash
.venv\Scripts\Activate.ps1
```

> **Finnes ikke .venv-mappen?**
>
> Dette skulle allerede vært satt opp tidligere, men du initialiserer enkelt
> et nytt venv-miljø slik:
>
> ```bash
> python -m venv .venv
> ```
>
> Så installerer du pydantic med:
>
> ```bash
> pip install pydantic
> ```

Sjekk at riktig miljø er valgt for denne jupyter notebooken. Det ser du øverst i
høyre hjørne av dette vinduet. Der skal det stå _.venv (Python 3.x.x)_ (x.x er
tall på versjonen du har installert). Hvis det ikke står _.venv_ der, eller det
står _Select kernel_ kan du klikke på teksten og får da opp valg om hvilket
miljø du skal bruke. Velg .venv miljøet.

Da skal du kunne importere `pydantic`! Kjør første kodecelle (over) på ny, og sjekk
at du får meldingen "pydantic er importert og klar til bruk".


***

## 3.2.1: Hva er Pydantic?

Pydantic er et Python-bibliotek for datavalidering og håndtering av
innstillinger. Det bruker Python sine type hints for å validere data automatisk
og gir deg klare feilmeldinger når noe er galt.

## 3.2.2: Hvorfor bruke Pydantic?

Pydantic har innebygd funksjonalitet for konvertering og validering, og
gjør det enklere å skrive robust kode for håndtering av data. Se eksempelet
under for manuelt kodet validering vs validering av data med pydantic.

### Uten Pydantic

```python
# Du må validere alt manuelt
data = {"name": "Ola", "age": "30"}  # age er string, ikke int!

if not isinstance(data["name"], str):
    raise ValueError("Name må være en string")
if not isinstance(data["age"], int):
    raise ValueError("Age må være et tall")
# ... og så videre for alle felt
```

### Med Pydantic

```python
from pydantic import BaseModel

class User(BaseModel):
    name: str
    age: int

# Pydantic validerer automatisk!
user = User(name="Ola", age="30")  # Konverterer "30" til 30
print(user.age)  # 30 (int)

# Gir feilmelding hvis data er feil
try:
    user = User(name="Ola", age="tretti")  # Kan ikke konverteres
except ValueError as e:
    print(e)  # Klar feilmelding om hva som er galt
```


## 3.2.3: Grunnleggende bruk av pydantic

### A) Definere en modell

Modeller defineres på ganske lik måte som `dataclass`, men i stedet
for å bruke en dekoratør før klassedefinisjonen, oppretter du klasser
som en underklasse av `BaseModel`. Din klasse vil da arve funksjonalitet
fra `BaseModel`.

```python
from pydantic import BaseModel

class Person(BaseModel):
    navn: str
    alder: int
    epost: str
```

### B) Opprette objekter

For å opprette instanser av modellen, gjør du akkurat som med vanlige
klasser:

```python
# Fra keyword arguments
person = Person(navn="Kari", alder=25, epost="kari@example.com")

# Fra dictionary
data = {"navn": "Ola", "alder": 30, "epost": "ola@example.com"}
person = Person(**data)
```


***

## 3.2.4: `Kontakt` som `pydantic`-modell

Nå skal vi se på hvordan vi kan definere `Kontakt` som en `pydantic`-modell.


In [ ]:
class Adresse(BaseModel):
    gate: str
    husnummer: str
    postnummer: int
    poststed: str
    land: str | None = None


class Kontakt(BaseModel):
    fornavn: str
    etternavn: str
    telefon: int | None = None
    mobil: int | None = None
    adresse: Adresse | None = None
    postnummer: int | None = None
    poststed: str | None = None
    epost: str | None = None


kontakt = Kontakt(
    fornavn="Eva",
    etternavn="Nilsen",
    telefon=33445566,
    mobil=99887766,
    adresse=Adresse(
        gate="Hovedgata",
        husnummer="10B",
        postnummer=1234,
        poststed="Oslo",
        land="Norge",
    ),
    epost="eva.nilsen@example.com",
)

print(kontakt)

Dette ser jo veldig likt ut som det vi gjorde tidligere? Ja, det er nesten likt!
I stedet for å bruke dekoratøren `@dataclass` arver vi fra klassen `BaseModel` i
`pydantic` ved å sette denne klassen i parantes bak klassenavnet i klassens hode:
```python
class MinPydanticModell(BaseClass):
```

Selv om det ser likt ut, har klasser som arver fra `BaseClass` mange nyttige
egenskaper og funksjoner. La oss se hva som skjer om vi prøver å gjøre en feil i
input av data:


In [ ]:
kontakt_feil_datatype = Kontakt(
    fornavn="Eva",
    etternavn="Nilsen",
    telefon="33445566",  # Feil datatype: telefon skal være int, ikke str
    mobil=99887766,
)

print("Utskrift av kontakt med feil datatype i initialisering:")
print(kontakt_feil_datatype)
print()
print(
    f"Datatype til telefon {kontakt_feil_datatype.telefon} er:",
    type(kontakt_feil_datatype.telefon),
)

La oss også prøve en annen type feil, der vi initialiserer kontakt med en verdi som hverken er int eller tall formatert som tekst:


In [ ]:
kontakt_feil_datatype = Kontakt(
    fornavn="Eva",
    etternavn="Nilsen",
    telefon="ikke_tallverdi",  # Feil verdi: ikke et tall
    mobil=99887766,
)



## Oppgave 3.2-1

Hva skjedde når vi initialiserte vår nye `Kontakt`-klasse med:
- Gyldig verdi, men feil datatype?
- Ugyldig verdi og datatype?

Hva forteller dette deg om `BaseModel` ?


_Skriv ditt svar her_
